# TODO: need to make sure the LiDAR download code works now

## Sample Bridge Decks Using USGS LiDAR (OPTIONAL)
---
Use this notebook to resample bridge deck elevations for specified network links

In [ ]:
import geopandas as gpd
import contextily as cx
import pandas as pd
import rasterio
from rasterio.features import geometry_mask
from rasterio.plot import show
import numpy as np
from shapely.geometry import box, mapping
from shapely.ops import Point
from tqdm import tqdm
from shapely.ops import LineString
import pickle

from bikewaysim.paths import config
from bikewaysim.network import elevation_tools

In [ ]:
links = gpd.read_file(config['network_fp'] / "osm.gpkg",layer="raw")
links.set_index('osmid',inplace=True)
lidar_urls = elevation_tools.get_lidar_urls(links)
print(len(lidar_urls),'LAZ files for the provided network')
print("Names:",set([lidar_url[1].split('/')[-3] for lidar_url in lidar_urls]))

100%|██████████| 9/9 [00:04<00:00,  2.05it/s]

461 LAZ files for the provided network
Names: {'GA_DEKALBCO_2010', 'GA_Statewide_B2_2018', 'GA_FULTONCO_2006', 'GA_Statewide_B3_2018'}


In [ ]:
# for the study area it's GA_Statewide_B2_2018 and GA_Statewide_B3_2018
lidar_urls = [url[1] for url in lidar_urls if 'GA_Statewide' in url[0]]

In [ ]:
# bridge_decks = elevation_tools.get_bridge_decks(lidar_urls)

Bridge deck found (1/244)
Error for https://rockyweb.usgs.gov/vdelivery/Datasets/Staged/Elevation/LPC/Projects/GA_Statewide_2018_B18_DRRA/GA_Statewide_B2_2018/LAZ/USGS_LPC_GA_Statewide_2018_B18_DRRA_e1062n1255.laz: HTTPSConnectionPool(host='rockyweb.usgs.gov', port=443): Read timed out.
Failed to download https://rockyweb.usgs.gov/vdelivery/Datasets/Staged/Elevation/LPC/Projects/GA_Statewide_2018_B18_DRRA/GA_Statewide_B2_2018/LAZ/USGS_LPC_GA_Statewide_2018_B18_DRRA_e1062n1255.laz after 10 retries.
Error for https://rockyweb.usgs.gov/vdelivery/Datasets/Staged/Elevation/LPC/Projects/GA_Statewide_2018_B18_DRRA/GA_Statewide_B2_2018/LAZ/USGS_LPC_GA_Statewide_2018_B18_DRRA_e1065n1253.laz: HTTPSConnectionPool(host='rockyweb.usgs.gov', port=443): Read timed out.
Failed to download https://rockyweb.usgs.gov/vdelivery/Datasets/Staged/Elevation/LPC/Projects/GA_Statewide_2018_B18_DRRA/GA_Statewide_B2_2018/LAZ/USGS_LPC_GA_Statewide_2018_B18_DRRA_e1065n1253.laz after 10 retries.
Bridge deck found (4

KeyboardInterrupt: 

In [ ]:
bridge_decks

In [ ]:
with (config['network_fp']/'bridge_deck_elevations.pkl').open('wb') as fh:
    pickle.dump(bridge_decks,fh)

# Sample Bridges from LiDAR

In [ ]:
with (config['network_fp']/'elevation.pkl').open('rb') as fh:
    interpolated_points_dict = pickle.load(fh)

In [ ]:
with (config['usgs_fp']/'lidar_points.pkl').open('rb') as fh:
    bridge_decks = pickle.load(fh)
bridge_decks.to_crs(dem_crs,inplace=True)
# # with (config['network_fp']/'bridge_deck_elevations.pkl').open('rb') as fh:
#     bridge_decks = pickle.load(fh)
# bridge_decks.to_crs(dem_crs,inplace=True)

In [ ]:
bridge_linkids = links[links['bridge'].notna()].index.tolist()
elevation_tools.sample_lidar(bridge_linkids,interpolated_points_dict,bridge_decks,dem_crs)

In [ ]:
#for storing the interpolated points with sampled elevation data
import pickle
with (config['network_fp']/'elevation.pkl').open('wb') as fh:
    pickle.dump(interpolated_points_dict,fh)

## Inspect LiDAR results

In [ ]:
with_lidar = {key:item['lidar'] for key,item in interpolated_points_dict.items() if 'lidar' in item.keys()}
with_lidar = [key for key,item in with_lidar.items() if np.isnan(item).all() == False]
links.reset_index(inplace=True)

In [ ]:
import random
from importlib import reload
reload(elevation_tools)
#pick some examples to visaulize here
# linkid = random.choice(with_lidar) # 
linkid = 38976112
grade_threshold = 20
elevation_tools.visualize(links,dem_crs,interpolated_points_dict,[linkid],grade_threshold,None,config['maptilerapikey'],one_off=True,lidar=True)